In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

random.seed(2)  # Set the seed for Python's random module
np.random.seed(3)  # Set the seed for numpy's random module

In [2]:
from cabm import cabm_model

In [9]:
model = cabm_model.ConsumerModel(1000, "config.toml", enable_ads=True, enable_pricepoint=False)

/Users/michael/Documents/github/consumer-abm/cabm/ad_calendar.py:78: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(0)
/Users/michael/Documents/github/consumer-abm/cabm/ad_calendar.py:78: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(0)


In [10]:
num_steps = 260

for i in range(num_steps):
    model.step()

In [11]:
agent_df = model.datacollector.get_agent_vars_dataframe()

In [12]:
# Chain copies of the 52 week plan together  to cover multi-year simulation (see note in TODO.txt about making this more flexible and robust)

# Prep joint calendar for plotting
joint_calendar_df = model.config.joint_calendar

joint_calendar_df['TV_A'] = joint_calendar_df[('A', 'TV')]
joint_calendar_df['Web_A'] = joint_calendar_df[('A', 'Web')]
joint_calendar_df['TV_B'] = joint_calendar_df[('B', 'TV')]
joint_calendar_df['Web_B'] = joint_calendar_df[('B', 'Web')]

# Create a date range starting from January 2021
date_range = pd.date_range(start='1/1/2021', periods=num_steps, freq='W')

# Repeat the joint_calendar_df to match the length of model_df
joint_calendar_df = pd.concat([joint_calendar_df]*int(len(date_range)/len(joint_calendar_df)), ignore_index=True)

# Add the date range as a new column in joint_calendar_df
joint_calendar_df['Date'] = date_range

In [13]:
agent_df.to_pickle('agent_output_A_ADS_ONLY.pkl')
joint_calendar_df.to_pickle('joint_calendar_A_ADS_ONLY.pkl')